In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
pip install pickle5

In [ ]:
import json 
import pandas as pd
import requests
import datetime
from collections import defaultdict
import pickle5 as pickle
import os
import numpy as np

path = "/content/drive/My Drive/winter2022/Projects/Covid19Tracker"

In [ ]:
import pandas as pd
import numpy as np
import requests
from plotly.offline import iplot
import plotly.graph_objs as go
import plotly.express as px
import json 
import datetime
from collections import defaultdict
import pickle5
import os
import warnings
from IPython.display import Markdown as md
from pytz import timezone

In [ ]:
def get_dates_till_today(start_date):
  return pd.date_range(end=datetime.datetime.today(), start=start_date).strftime('%Y-%m-%d').tolist()

def get_region_report(region_name, date, df):
  url = "https://covid-19-statistics.p.rapidapi.com"
  headers = {
      'x-rapidapi-host': 'covid-19-statistics.p.rapidapi.com',
      'x-rapidapi-key': '44e824ba26mshbd88513db1a43d3p174f66jsn2f60602ce307'
    }
  #Get countries names and iso
  response = requests.request(method="GET", url=f"{url}/reports", 
              params={'iso': df.loc[df.name == region_name, 'iso'], 
                'region_name': region_name, 'date': date}, headers=headers)
  response_json = json.loads(response.text)

  if len(response_json) > 0 and 'data' in response_json:
    return response_json['data']

  return []

def get_worldwide_data(date):
    url = "https://covid-19-statistics.p.rapidapi.com"
    headers = {
        'x-rapidapi-host': 'covid-19-statistics.p.rapidapi.com',
        'x-rapidapi-key': '44e824ba26mshbd88513db1a43d3p174f66jsn2f60602ce307'
    }
    #Get countries names and iso
    response = requests.request(method="GET", url=f"{url}/reports/total",params={'date':date}, headers=headers)
    #json.loads(response.text)['data']
    response_json = json.loads(response.text)

    if len(response_json) > 0 and 'data' in response_json:
      return response_json['data']

    return []

def get_latest_worldwide_data():
    url = "https://covid-19-statistics.p.rapidapi.com"
    headers = {
        'x-rapidapi-host': 'covid-19-statistics.p.rapidapi.com',
        'x-rapidapi-key': '44e824ba26mshbd88513db1a43d3p174f66jsn2f60602ce307'
    }
    #Get countries names and iso
    response = requests.request(method="GET", url=f"{url}/reports/total", headers=headers)
    #json.loads(response.text)['data']
    response_json = json.loads(response.text)

    if len(response_json) > 0 and 'data' in response_json:
      return response_json['data']

    return []

def get_countries_data(dates, regions, df_regions):
  temp = defaultdict(list)
  result = defaultdict(dict)
  for region in regions:
    for date in dates:
      sol = {'confirmed_diff':0, 'deaths_diff':0, 'date':date}
      data = get_region_report(region, date, df_regions)
      if len(data)> 0:
        for report in data:
          sol['confirmed_diff'] += report['confirmed_diff']
          sol['deaths_diff'] += report['deaths_diff']
      temp[region].append(sol)

  for region in regions:
    result[region]['confirmed_diff'] = np.array([sub['confirmed_diff'] for sub in temp[region]])
    result[region]['deaths_diff'] = np.array([sub['deaths_diff'] for sub in temp[region]])
  result['dates'] = np.array(dates)

  #Get global data and append it to the result
  glob_data = defaultdict(list)
  for date in dates:
    data = get_worldwide_data(date)
    if len(data)>0:
      glob_data['global_conf_diff'].append(data['confirmed_diff'])
      glob_data['global_deaths_diff'].append(data['deaths_diff'])

  result['global_conf_diff'] = np.array(glob_data['global_conf_diff'])
  result['global_deaths_diff'] = np.array(glob_data['global_deaths_diff'])

  return result

def get_latest_weekly_data(data, regions, n_weeks):
  sol = {'conf_per_rgn': np.array([np.sum(data[rgn]['confirmed_diff'][-(7*n_weeks):]) for rgn in regions]), 'deaths_per_rgn': np.array([np.sum(data[rgn]['deaths_diff'][-(7*n_weeks):]) for rgn in regions])}
  return sol

def get_optimized_regions_data(data, regions):
  result = data.copy()
  #Get new cases globally and per region
  #New cases equals to the average of confirmed cases over the last 7 days
  result['global_new_cases'] = np.array([np.mean(result['global_conf_diff'][-8:-1]), 
  np.mean(result['global_conf_diff'][-7:])])
  result['new_cases_per_rgn'] = {k:np.mean(result[k]['confirmed_diff'][-7:]) for k in regions}
  result['top5_rgn_new_cases'] = sorted(result['new_cases_per_rgn'].items(), key=lambda x: x[1], reverse=True)[:5]

  #Get the data for the latest 1 to 5 weeks
  for i in range(1,6):
    result[f"top{i}_wk"] = get_latest_weekly_data(result, regions, i)
  
  return result.copy()

def update_cached_data(data, new_data, regions):
  old_data = data.copy()
  for rgn in regions:
    old_data[rgn]['confirmed_diff'] = np.append(old_data[rgn]['confirmed_diff'][:-1], 
    new_data[rgn]['confirmed_diff'])
    old_data[rgn]['deaths_diff'] = np.append(old_data[rgn]['deaths_diff'][:-1], 
    new_data[rgn]['deaths_diff'])

  old_data['dates'] = np.append(old_data['dates'][:-1], new_data['dates'])
  old_data['global_conf_diff'] = np.append(old_data['global_conf_diff'][:-1], new_data['global_conf_diff'])
  old_data['global_deaths_diff'] = np.append(old_data['global_deaths_diff'][:-1], 
  new_data['global_deaths_diff'])

  return old_data.copy()

In [ ]:
#Initialize the starting date in our data
initial_date = datetime.datetime.today()
dates = get_dates_till_today(initial_date)

def get_regions_list():
    url = "https://covid-19-statistics.p.rapidapi.com"
    headers = {
        'x-rapidapi-host': 'covid-19-statistics.p.rapidapi.com',
        'x-rapidapi-key': '96a36f8da6msh9ee894aabc7ae5bp196263jsn8dfae3d97a64'
    }
    #Get countries names and iso
    response = requests.request(method="GET", url=f"{url}/regions", headers=headers)
    #json.loads(response.text)['data']
    response_json = json.loads(response.text)['data']

    return pd.json_normalize(response_json)

df_regions = get_regions_list()
df_regions = df_regions.sort_values('name').reset_index(drop=True)
regions = list(df_regions.name)
#Clean the list of regions
regions = [x for x in regions if x not in ['Others','Cruise Ship']]
iso = [df_regions.loc[df_regions.name==rgn, 'iso'].values[0] for rgn in regions]


In [ ]:
def cache_data(data, latest_date, regions, df_regions):
    time1 = datetime.datetime.strptime(datetime.datetime.now(timezone('Canada/Eastern')).strftime('%Y-%m-%d'), '%Y-%m-%d')
    if time1 > (latest_date + datetime.timedelta(days=1)):
        new_dates = get_dates_till_today(latest_date)
        #Remove today's date from the dates to be cached
        del new_dates[-1]
        if len(new_dates)>0:
            new_data = get_countries_data(new_dates, regions, df_regions)
            #Update the data
            #Debugging !!!!
            #st.write("Caching the data!!!")

            temp = update_cached_data(data, new_data, regions)
            data = get_optimized_regions_data(temp, regions)
            #Cache the new data
            with open(f"{path}/data/cache_final.p", 'wb') as fp:
                pickle.dump(data, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
def load_cache(filepath=f"{path}/data/cache_final.p"):
    #Check first if there is a cached file and if it can be accessed
    if os.path.isfile(filepath) and os.access(filepath, os.R_OK):
        with open(filepath, 'rb') as fp:
            return pickle5.load(fp)
    else:
        return {}

cached_data = load_cache()

#Use the first date in the data to cache all data
latest_cache_date = initial_date
data = dict()

In [ ]:
cached_data['dates'][-5:]

array(['2022-01-21', '2022-01-22', '2022-01-23', '2022-01-24',
       '2022-01-25'], dtype='<U10')

In [ ]:
#Use the first date in the data to cache all data
latest_cache_date = initial_date
data = dict()

if len(cached_data)>0:
    #Get the latest cache date
    data = cached_data.copy()
    temp = data['dates'][-1]
    latest_cache_date = datetime.datetime.strptime(temp, '%Y-%m-%d')
    #Update the initial date to the first date in our cached data
    initial_date = data['dates'][0]
    initial_date = datetime.datetime.strptime(initial_date, '%Y-%m-%d')
    dates = get_dates_till_today(initial_date)
    #Update cached data  
    cache_data(data, latest_cache_date, regions, df_regions)
else:
    print("Error, No cache found!!!")

In [ ]:
def get_regions_list():
    url = "https://covid-19-statistics.p.rapidapi.com"
    headers = {
        'x-rapidapi-host': 'covid-19-statistics.p.rapidapi.com',
        'x-rapidapi-key': '44e824ba26mshbd88513db1a43d3p174f66jsn2f60602ce307'
    }
    #Get countries names and iso
    response = requests.request(method="GET", url=f"{url}/regions", headers=headers)
    #json.loads(response.text)['data']
    response_json = json.loads(response.text)['data']

    return pd.json_normalize(response_json)

df_regions = get_regions_list()
df_regions = df_regions.sort_values('name').reset_index(drop=True)
regions = list(df_regions.name)
#Clean the list of regions
regions = [x for x in regions if x not in ['Others','Cruise Ship']]
iso = [df_regions.loc[df_regions.name==rgn, 'iso'].values[0] for rgn in regions]

In [ ]:
def get_latest_worldwide_data():
    url = "https://covid-19-statistics.p.rapidapi.com"
    headers = {
        'x-rapidapi-host': 'covid-19-statistics.p.rapidapi.com',
        'x-rapidapi-key': '44e824ba26mshbd88513db1a43d3p174f66jsn2f60602ce307'
    }
    #Get countries names and iso
    response = requests.request(method="GET", url=f"{url}/reports/total", headers=headers)
    #json.loads(response.text)['data']
    response_json = json.loads(response.text)

    if len(response_json) > 0:
      return response_json['data']

    return []
tt = get_latest_worldwide_data()

In [ ]:
datetime.datetime.strptime(tt['last_update'], '%Y-%m-%d %H:%M:%S').strftime('%A %B %d, %Y at %I:%M %p')

'Thursday January 13, 2022 at 08:00 AM'

In [ ]:
def get_region_report(region_name, date, df=df_regions):
  url = "https://covid-19-statistics.p.rapidapi.com"
  headers = {
      'x-rapidapi-host': 'covid-19-statistics.p.rapidapi.com',
      'x-rapidapi-key': '44e824ba26mshbd88513db1a43d3p174f66jsn2f60602ce307'
    }
  #Get countries names and iso
  response = requests.request(method="GET", url=f"{url}/reports", 
              params={'iso': df.loc[df.name == region_name, 'iso'], 
                'region_name': region_name, 'date': date}, headers=headers)
  response_json = json.loads(response.text)

  if len(response_json) > 0:
    return response_json['data']

  return []


def get_countries_data(dates, regions=regions):
  temp = defaultdict(list)
  result = defaultdict(dict)
  for region in regions:
    for date in dates:
      sol = {'confirmed_diff':0, 'deaths_diff':0, 'date':date}
      data = get_region_report(region, date)
      if len(data)> 0:
        for report in data:
          sol['confirmed_diff'] += report['confirmed_diff']
          sol['deaths_diff'] += report['deaths_diff']
      temp[region].append(sol)

  for region in regions:
    result[region]['confirmed_diff'] = np.array([sub['confirmed_diff'] for sub in temp[region]])
    result[region]['deaths_diff'] = np.array([sub['deaths_diff'] for sub in temp[region]])
  result['dates'] = np.array(dates)

  #Get global data and append it to the result
  glob_data = defaultdict(list)
  for date in dates:
    data = get_worldwide_data(date)
    glob_data['global_conf_diff'].append(data['confirmed_diff'])
    glob_data['global_deaths_diff'].append(data['deaths_diff'])

  result['global_conf_diff'] = np.array(glob_data['global_conf_diff'])
  result['global_deaths_diff'] = np.array(glob_data['global_deaths_diff'])

  return result

def get_latest_weekly_data(data, regions, n_weeks):
  sol = {'conf_per_rgn': np.array([np.sum(data[rgn]['confirmed_diff'][-(7*n_weeks):]) for rgn in regions]),  
          'deaths_per_rgn': np.array([np.sum(data[rgn]['deaths_diff'][-(7*n_weeks):]) for rgn in regions])}
  return sol

def get_optimized_regions_data(data, regions=regions):
  result = data.copy()
  #Get new cases globally and per region
  #New cases equals to the average of confirmed cases over the last 7 days
  result['global_new_cases'] = np.array([np.mean(result['global_conf_diff'][-8:-1]), np.mean(result['global_conf_diff'][-7:])])
  result['new_cases_per_rgn'] = {k:np.mean(result[k]['confirmed_diff'][-7:]) for k in regions}
  result['top5_rgn_new_cases'] = sorted(result['new_cases_per_rgn'].items(), key=lambda x: x[1], reverse=True)[:5]

  #Get the data for the latest 1 to 5 weeks
  for i in range(1,6):
    result[f"top{i}_wk"] = get_latest_weekly_data(result, regions, i)
  
  return result.copy()

def update_cached_data(data, new_data, regions=regions):
  old_data = data.copy()
  for region in regions:
    old_data[rgn]['confirmed_diff'] = np.append(old_data[rgn]['confirmed_diff'][:-1], new_data[rgn]['confirmed_diff'])
    old_data[rgn]['deaths_diff'] = np.append(old_data[rgn]['deaths_diff'][:-1], new_data[rgn]['deaths_diff'])

  old_data['dates'] = np.append(old_data['dates'][:-1], new_data['dates'])
  old_data['global_conf_diff'] = np.append(old_data['global_conf_diff'][:-1], new_data['global_conf_diff'])
  old_data['global_deaths_diff'] = np.append(old_data['global_deaths_diff'][:-1], new_data['global_deaths_diff'])

  return old_data.copy()



def get_dates_till_today(start_date):
  return pd.date_range(end='2022-01-10', start=start_date).strftime('%Y-%m-%d').tolist()

dates = get_dates_till_today('2021-11-01')



In [ ]:
list(data.keys())[-11:]

['dates',
 'global_conf_diff',
 'global_deaths_diff',
 'global_new_cases',
 'new_cases_per_rgn',
 'top5_rgn_new_cases',
 'top1_wk',
 'top2_wk',
 'top3_wk',
 'top4_wk',
 'top5_wk']

In [ ]:
countries = ['Albania','US']
chart_data = [data[x]['confirmed_diff'] for x in countries]
df = pd.DataFrame(chart_data).transpose()
df.columns = countries
df

,Albania,US
0,197,121139
1,725,77088
2,571,83501
3,570,82343
4,631,101111
...,...,...
69,4789,305100
70,793,1485764
71,793,1485764
72,793,1485764


In [ ]:
import plotly.express as px

df = px.data.gapminder().query("continent=='Oceania'")
df.head()

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
60,Australia,Oceania,1952,69.12,8691212,10039.59564,AUS,36
61,Australia,Oceania,1957,70.33,9712569,10949.64959,AUS,36
62,Australia,Oceania,1962,70.93,10794968,12217.22686,AUS,36
63,Australia,Oceania,1967,71.10,11872264,14526.12465,AUS,36
64,Australia,Oceania,1972,71.93,13177000,16788.62948,AUS,36


In [ ]:
data['US'].keys()

dict_keys(['confirmed_diff', 'deaths_diff'])

In [ ]:
data = load_cache(f"{path}/data/cache_final.p")

In [ ]:
import datetime
temp = data['dates'][-1]
latest_cache_date = datetime.datetime.strptime(temp, '%Y-%m-%d')

(datetime.datetime.strptime(datetime.datetime.now(timezone('Canada/Eastern')).strftime('%Y-%m-%d'), '%Y-%m-%d') > 
 (latest_cache_date + datetime.timedelta(days=1)))

False

In [ ]:
list(data.keys())[-11:]

['dates',
 'global_conf_diff',
 'global_deaths_diff',
 'global_new_cases',
 'new_cases_per_rgn',
 'top5_rgn_new_cases',
 'top1_wk',
 'top2_wk',
 'top3_wk',
 'top4_wk',
 'top5_wk']

In [ ]:
data['top5_rgn_new_cases']

[('US', 970285.0),
 ('France', 225521.7142857143),
 ('Spain', 179223.42857142858),
 ('Italy', 147744.0),
 ('United Kingdom', 145149.42857142858)]

In [ ]:
from pytz import timezone
datetime.datetime.now(timezone('Canada/Eastern')).strftime('%Y-%m-%d') > temp

True

In [ ]:
data['dates'] = np.array(get_dates_till_today('2021-11-01'))

In [ ]:
data['US'].keys()

dict_keys(['confirmed_diff', 'deaths_diff'])

In [ ]:
loaded_data = load_cache(f"{path}/data/cache_clean.p")
#Remove ['Others','Cruise Ship'] from loaded data
for k in ['Others','Cruise Ship']:
  del loaded_data[k]
print(len(loaded_data))

216


In [ ]:
#Cache the latest version of data from 2021-11-01 to 2022-01-10
cache_data(data)

In [ ]:
#Add gobal data to loaded data
loaded_data['global_conf_diff'] = result['global_conf_diff']
loaded_data['global_deaths_diff'] = result['global_deaths_diff']
#New cases equals to the average of confirmed cases over the last 7 days
loaded_data['global_new_cases'] = [np.mean(loaded_data['global_conf_diff'][-8:-1]), np.mean(loaded_data['global_conf_diff'][-7:])]
loaded_data['new_cases_per_rgn'] = {k:np.mean(loaded_data[k]['confirmed_diff'][-7:]) for k in regions}
loaded_data['top5_rgn_new_cases'] = sorted(loaded_data['new_cases_per_rgn'].items(), key=lambda x: x[1], reverse=True)[:5]
#Get the data for the latest 1 to 5 weeks
for i in range(1,6):
  loaded_data[f"top{i}_wk"] = get_latest_weekly_data(loaded_data, regions, i)


In [ ]:
def get_worldwide_data(date):
    url = "https://covid-19-statistics.p.rapidapi.com"
    headers = {
        'x-rapidapi-host': 'covid-19-statistics.p.rapidapi.com',
        'x-rapidapi-key': '44e824ba26mshbd88513db1a43d3p174f66jsn2f60602ce307'
    }
    #Get countries names and iso
    response = requests.request(method="GET", url=f"{url}/reports/total",params={'date':date}, headers=headers)
    #json.loads(response.text)['data']
    response_json = json.loads(response.text)

    if len(response_json) > 0:
      return response_json['data']

    return []

In [ ]:
yesterday_date = (datetime.datetime.today()-datetime.timedelta(days=1)).strftime('%Y-%m-%d')
get_worldwide_data(yesterday_date)

{'active': 307959123,
 'active_diff': 3140243,
 'confirmed': 313463348,
 'confirmed_diff': 3149620,
 'date': '2022-01-11',
 'deaths': 5504225,
 'deaths_diff': 9377,
 'fatality_rate': 0.0176,
 'last_update': '2022-01-12 04:21:44',
 'recovered': 0,
 'recovered_diff': 0}

In [ ]:
last_update = datetime.datetime.strptime(get_worldwide_data(yesterday_date)['last_update'], "%Y-%m-%d %H:%M:%S")
last_update.strftime('%A %B %d, %Y at %I:%M %p')



'Wednesday January 12, 2022 at 04:21 AM'

In [ ]:
datetime.datetime.strptime(, "%Y-%m-%d")

In [ ]:
ww_dates = get_dates_till_today('2021-11-01')
result = defaultdict(list)
for date in ww_dates:
  data = get_worldwide_data(date)
  result['global_conf_diff'].append(data['confirmed_diff'])
  result['global_deaths_diff'].append(data['deaths_diff'])
result

defaultdict(list,
            {'global_conf_diff': [429078,
              444557,
              506137,
              526304,
              527698,
              404214,
              328200,
              491034,
              478090,
              571367,
              534372,
              572515,
              445068,
              362037,
              539814,
              511049,
              620221,
              1069082,
              630160,
              466500,
              389081,
              721490,
              523767,
              672494,
              588692,
              596722,
              415822,
              429441,
              678657,
              623124,
              703436,
              701520,
              721825,
              447485,
              482992,
              594677,
              701169,
              669745,
              729521,
              618428,
              559938,
              419474,
              630403,
              6

In [ ]:
get_worldwide_data('2021-11-01')['confirmed_diff']

429078

In [ ]:
def load_cache(filepath):
  #Check first if there is a cached file and if it can be accessed
  if os.path.isfile(filepath) and os.access(filepath, os.R_OK):
    with open(filepath, 'rb') as fp:
      return pickle5.load(fp)
    

def cache_data(data):
  #if (datetime.datetime.today()-datetime.timedelta(days=1)) > datetime.datetime.strptime(latest_date, '%Y-%m-%d'):
  #new_dates = get_dates_till_today(latest_date)
  #if len(new_dates)>0:
  cache = data.copy()
  with open(f"{path}/data/cache_final.p", 'wb') as fp:
    pickle.dump(cache, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
data = get_countries_data(regions, dates)

cache_data(data)

In [ ]:
cached_data_v1 = load_cache(f"{path}/data/cache_prev.p")
cached_data_v2 = load_cache(f"{path}/data/cache.p")

In [ ]:
clean_data = dict()
for region in regions:
  t1 = cached_data_v1[region] + [data_v1[region][-1]]
  t2 = cached_data_v2[region][:-1] + data_v2[region]
  clean_data[region] = t1 + t2

In [ ]:
get_region_dataframe('US', clean_data)

,confirmed,deaths,date,confirmed_diff,year_week,new_cases,min_confirmed,max_confirmed,min_deaths,max_deaths
0,59767221,837262,2022-01-08,378692,"(2022, 1)",775527.333333,56190946,59767221,827753,837262
1,59388529,836603,2022-01-07,900864,"(2022, 1)",775527.333333,56190946,59767221,827753,837262
2,58487665,833987,2022-01-06,785288,"(2022, 1)",775527.333333,56190946,59767221,827753,837262
3,57702377,832118,2022-01-05,653577,"(2022, 1)",775527.333333,56190946,59767221,827753,837262
4,57048800,830071,2022-01-04,857854,"(2022, 1)",775527.333333,56190946,59767221,827753,837262
...,...,...,...,...,...,...,...,...,...,...
64,46435967,753926,2021-11-05,101111,"(2021, 44)",73858.000000,46091924,46487791,747033,754429
65,46334856,751555,2021-11-04,82343,"(2021, 44)",73858.000000,46091924,46487791,747033,754429
66,46252513,750423,2021-11-03,83501,"(2021, 44)",73858.000000,46091924,46487791,747033,754429
67,46169012,748518,2021-11-02,77088,"(2021, 44)",73858.000000,46091924,46487791,747033,754429


In [ ]:
def get_region_dataframe(region, data):
  df = pd.DataFrame(data[region])
  #For the sake of fixing the latest data val
  #df = df.iloc[:-1, :].reset_index(drop=True)
  df["year_week"] = df.apply(lambda x: datetime.datetime.strptime(x['date'], "%Y-%m-%d").isocalendar()[:2], axis=1)
  df_new_cases = df.groupby("year_week", as_index=False)["confirmed_diff"].mean().rename(columns={'confirmed_diff': 'new_cases'})
  df = df.merge(df_new_cases, how='left', on='year_week')
  df['min_confirmed'] = df.groupby('year_week')['confirmed'].transform('min')
  df['max_confirmed'] = df.groupby('year_week')['confirmed'].transform('max')
  df['min_deaths'] = df.groupby('year_week')['deaths'].transform('min')
  df['max_deaths'] = df.groupby('year_week')['deaths'].transform('max')
  
  return df

#test_v1 = get_region_dataframe("US", cached_data_v1)


In [ ]:
result.keys()

dict_keys(['2021-11-01', '2021-11-02', '2021-11-03', '2021-11-04', '2021-11-05', '2021-11-06', '2021-11-07', '2021-11-08', '2021-11-09', '2021-11-10', '2021-11-11', '2021-11-12', '2021-11-13', '2021-11-14', '2021-11-15', '2021-11-16', '2021-11-17', '2021-11-18', '2021-11-19', '2021-11-20', '2021-11-21', '2021-11-22', '2021-11-23', '2021-11-24', '2021-11-25', '2021-11-26', '2021-11-27', '2021-11-28', '2021-11-29', '2021-11-30', '2021-12-01', '2021-12-02', '2021-12-03', '2021-12-04', '2021-12-05', '2021-12-06', '2021-12-07', '2021-12-08', '2021-12-09', '2021-12-10', '2021-12-11', '2021-12-12', '2021-12-13', '2021-12-14', '2021-12-15', '2021-12-16', '2021-12-17', '2021-12-18', '2021-12-19', '2021-12-20', '2021-12-21', '2021-12-22', '2021-12-23', '2021-12-24', '2021-12-25', '2021-12-26', '2021-12-27', '2021-12-28', '2021-12-29', '2021-12-30', '2021-12-31', '2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04', '2022-01-05', '2022-01-06', '2022-01-07', '2022-01-08'])

In [ ]:
def get_clean_cache(detailed_data, global_data, regions=regions, dates=dates):
  #Clean the list of regions
  regions = [x for x in regions if x not in ['Others','Cruise Ship']]

  final_result = dict()
  list_key_data = ['confirmed','deaths']
  val_key_data = ['new_cases','min_confirmed','max_confirmed','min_deaths','max_deaths']

  for region in regions:
    region_df = get_region_dataframe(region, detailed_data)
    temp = defaultdict(dict)
    for yw in set(region_df.year_week):
      for key in list_key_data:
        temp[f"{yw}"][key] = list(region_df.loc[region_df.year_week==yw, key])
      for key in val_key_data:
        temp[f"{yw}"][key] = list(region_df.loc[region_df.year_week==yw, key])[0]
      
    final_result[region] = temp
  
  conf_global = [x['confirmed'] for x in global_data.values()]
  deaths_global = [x['deaths'] for x in global_data.values()]
  dates_global = list(global_data.keys())

  final_result['confirmed_global']=conf_global
  final_result['deaths_global']=deaths_global
  final_result['dates_global']=dates_global

  return final_result



In [ ]:
trial = get_clean_cache(clean_data, result)

In [ ]:
def get_new_cases(data, region):
  cached_weeks = sorted(data[region].keys())
  latest_week = cached_weeks[-1]
  prev_week = cached_weeks[-2]
  latest_week_length = len(data[region][latest_week]['confirmed'])
  prev_wk_idx = -1-(7-latest_week_length)

  latest_wk_max = data[region][latest_week]['max_confirmed']
  prev_week_val = data[region][prev_week]['confirmed'][prev_wk_idx]

  return (latest_wk_max-prev_week_val)/7

get_new_cases(trial, 'US')

701036.4285714285

In [ ]:
def get_region_req_data(data, region, n_weeks, case_type):
  cached_weeks = sorted(data[region].keys())
  result = list()
  i=n_weeks
  while i > 0:
    result.extend(data[region][cached_weeks[-i]][case_type])
    i=i-1
  return result

t1 =  get_region_req_data(trial, 'US', 2, 'confirmed')
dates_to_fetch = datetime.datetime.strptime(trial['dates_global'][-1], "%Y-%m-%d").isocalendar()[2]+ (num_weeks-1)*7
t2 = trial['confirmed_global'][-dates_to_fetch:]
print(t1)
print(t2)
print(len(t1)==len(t2))

[52793407, 53170421, 53659688, 54306755, 54743982, 54859966, 55114057, 56190946, 57048800, 57702377, 58487665, 59388529, 59767221]
[281393422, 282789956, 284516633, 286522415, 288172265, 289251016, 290111511, 292502565, 295149270, 297650862, 300172279, 302998523, 305000434]
True


In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')
df

,COUNTRY,GDP (BILLIONS),CODE
0,Afghanistan,21.71,AFG
1,Albania,13.40,ALB
2,Algeria,227.80,DZA
3,American Samoa,0.75,ASM
4,Andorra,4.80,AND
...,...,...,...
217,Virgin Islands,5.08,VGB
218,West Bank,6.64,WBG
219,Yemen,45.45,YEM
220,Zambia,25.61,ZMB


In [ ]:
def get_string_date_from_weeknum(data):
  d = f"{data[0]}-W{data[1]}"
  week_of = datetime.datetime.strptime(d + '-1', "%Y-W%W-%w").strftime('%Y/%m/%d')
  return week_of

def get_case_specific_df(case_type, data, n_weeks):

  regions_iso = {k:v for k,v in list(zip(df_regions['name'], df_regions['iso'])) if v not in ['Others','cruise']}
  home_data = defaultdict(list)
  
  for region in regions_iso.keys():
    temp_df = get_new_cases(n_weeks, get_region_dataframe(region, data))
    #Create a dictionary with data for all regions
    home_data['confirmed'].append(temp_df['confirmed'].sum())
    home_data['deaths'].append(temp_df['deaths'].sum())
    home_data['new_cases'].append(temp_df['new_cases'].sum())
  
  #Get iso codes and names
  iso_codes = list(regions_iso.values())
  region_names = list(regions_iso.keys())

  if case_type == 'confirmed':
    return region_names, iso_codes, home_data['confirmed']
  elif case_type == 'deaths':
    return region_names, iso_codes, home_data['deaths']
  elif case_type == 'new_cases':
    return region_names, iso_codes, home_data['new_cases']
  else:
    pass



In [ ]:
country_names, iso, case_type_data = get_case_specific_df('confirmed', cached_data)
weeks = get_case_specific_df('week_range', cached_data)

fig = go.Figure(data=go.Choropleth(
    locations = iso,
    z = case_type_data,
    text = country_names,
    colorscale = 'Blues',
    autocolorscale=False,
    reversescale=True,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_title = "Number of<br>COVID19 cases",
))

fig.update_layout(
    title_text= f"Number of COVID19 cases for the last {len(weeks)} weeks",
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    )
)

fig.show()

['VAT',
 'GGY-JEY',
 'MYT',
 'ESH',
 'RKS',
 'PSE',
 'REU',
 'GUF',
 'MTQ',
 'GLP',
 'NA-SHIP-MSZ',
 'BHS',
 'BLM',
 'NA-SHIP-DP']